In [1]:
import torch

In [2]:
# Define the shape of the tensor
b=100 #individuals
p=2 #ploidy
c=5 #chromosome
l = 10 #loci
def dummy_haplotypes(b,p,c,l): return torch.randint(0, 2, (b, p, c, l))

dummy_mating_pool = dummy_haplotypes(b,p,c,l)
#add batch dimension, single item per batch
# dummy_mating_pool.unsqueeze(0) #(1, 100, 2, 5, 10)
mating_pool_bv = torch.randn(b)
mating_pool_bv = mating_pool_bv.unsqueeze(0) # (1,100)

In [63]:
import torch
import torch.nn as nn
import torch.optim as optim

class CNN1D(nn.Module):
    def __init__(self):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=64, kernel_size=32, stride=8)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=16, kernel_size=8, stride=2)
        self.fc1 = nn.Linear(928, 64)
        self.fc2 = nn.Linear(64, 32)  # Changed to 32 to match the dimension of combined features

    def forward(self, x):
        x1 = self.conv1(x)
        x1 = nn.ReLU()(x1)
        x1 = self.conv2(x1)
        x1 = nn.ReLU()(x1)
        x1 = x1.view(x1.size(0), -1)

        x2 = self.conv1(x.flip(dims=[1]))
        x2 = nn.ReLU()(x2)
        x2 = self.conv2(x2)
        x2 = nn.ReLU()(x2)
        x2 = x2.view(x2.size(0), -1)

        x = (x1 + x2) / 2
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x
    
class MappingNetwork(nn.Module):
    def __init__(self):
        super(MappingNetwork, self).__init__()
        self.mapping = nn.Linear(1, 16)

    def forward(self, x):
        x = self.mapping(x)
        x = nn.ReLU()(x)
        return x
    
class CombinedModel(nn.Module):
    def __init__(self):
        super(CombinedModel, self).__init__()
        self.geno_net = CNN1D()
        self.meta_net = MappingNetwork()
        self.fc1 = nn.Linear(32 + 16, 64)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()  # Add sigmoid layer

    def forward(self, geno_input, meta_input):
        geno_features = self.geno_net(geno_input)
        meta_features = self.meta_net(meta_input)
        combined_features = torch.cat((geno_features, meta_features), dim=1)
        x = self.fc1(combined_features)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        x = self.sigmoid(x)  # Apply sigmoid activation
        return x

# Example usage:
input_tensor = torch.randn(1, 2, 1000)  # [batch_size, channels, length]
meta_tensor = torch.tensor([[1.0]])  # [batch_size, 1]

model = CombinedModel()
output = model(input_tensor, meta_tensor)
print(output)

tensor([[0.4972]], grad_fn=<SigmoidBackward0>)
